# Gousto Recipe Rank Forecasting: MSc Project

In [3]:
# Import all modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import os
import json
import shap

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
import optuna
import optuna.visualization as vis
import time

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout, Input, LSTM

In [4]:
# Data load
directory = os.getcwd()
file_path = os.path.join(directory, '..', 'data', 'extended_training_df_619.json')

# Read JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

fields = [field['name'] for field in data['schema']['fields'] if field['name'] != 'index']
records = data.get('data', [])

# Convert to Pandas DataFrame
df = pd.DataFrame(records, columns=fields)
df['item_id'] = pd.to_numeric(df['item_id'], errors='coerce')

In [5]:
# Data Transformations
df_full = df.copy()

# Remove short term model features of uptake at lead day and differences
columns_to_remove = df.filter(like="uptake_at_lead_day").columns
df.drop(columns=columns_to_remove, inplace=True)

# Remove contamination outlier feature as this is not present in test data
df.drop(columns=['contamination_outlier'], inplace=True)

# Create recipe rank percentile column (still called recipe rank)
df['recipe_rank'] = df.groupby('menu_week')['recipe_uptake'].rank(method='first', ascending=False) - 1
df['recipe_rank'] = df['recipe_rank'].astype(int)
# Drop recipe uptake column
df = df.drop(columns=['recipe_uptake'])

In [ ]:
# PCA
# Create df before PCA, removing target and item_id columns
df_pca_pre = df.drop(columns=['recipe_rank', 'item_id'])

# Standardize the data
scaler = StandardScaler()
df_pca_pre_scaled = scaler.fit_transform(df_pca_pre)

# Perform PCA to retain 95% variance
pca = PCA(n_components=0.95)
df_pca_transformed = pca.fit_transform(df_pca_pre_scaled)

# Convert the PCA result back to a DataFrame
pca_columns = [f'PC{i+1}' for i in range(df_pca_transformed.shape[1])]
df_pca = pd.DataFrame(df_pca_transformed, columns=pca_columns, index=df_pca_pre.index)

# Combine the results back with the untouched columns and menu_week
df_final = pd.concat([df_pca, df[['menu_week', 'item_id', 'recipe_rank']]], axis=1)

# Plot Explained Variance
explained_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, marker='o', linestyle='--')
plt.axhline(y=0.9, color='r', linestyle='--', label='90% Variance')
plt.axhline(y=0.95, color='g', linestyle='--', label='95% Variance')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance vs Components')
plt.legend(loc='best')
plt.grid(True)
plt.show()

In [7]:
# Function to normalise data
def normalize_data(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Convert back to DataFrame to maintain the index
    X_train_scaled = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)
    X_test_scaled = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns)
    
    return X_train_scaled, X_test_scaled

# Function for custom time series cross validation split, for the final 3 menu weeks
def custom_time_series_cv(X_train, n_validation_weeks=3):
    validation_weeks = X_train['menu_week'].unique()[-n_validation_weeks:]
    for valid_week in validation_weeks:
        train_df = X_train[X_train['menu_week'] < valid_week]
        valid_df = X_train[X_train['menu_week'] == valid_week]
        yield train_df.index, valid_df.index

In [8]:
# Function to evaluate predictions and plot error bars for statistical algorithms
def evaluate_predictions_algos(all_errors, method, weeks_before_target=8, total_week_iterations=20):
    min_y_range = 0.05
    average_errors, lower_bound_errors, upper_bound_errors = [], [], []

    for i in range(weeks_before_target):
        errors_for_week = [iteration[i]['mean_absolute_error'] for iteration in all_errors if i < len(iteration)]
        if errors_for_week:
            average_error = np.mean(errors_for_week)
            sem = np.std(errors_for_week) / np.sqrt(len(errors_for_week))
            ci = 1.96 * sem  # 95% confidence interval

            # Store results
            average_errors.append({'week_before_target': weeks_before_target - i, 'average_mean_absolute_error': average_error})
            lower_bound_errors.append(average_error - ci)
            upper_bound_errors.append(average_error + ci)

    print(f"Method: {method} | MAEs {average_errors}")
    # Convert to DataFrame for easier access
    average_errors_df = pd.DataFrame(average_errors)
    avg_errors_array = average_errors_df['average_mean_absolute_error'].to_numpy()

    # Calculate y-limits and add margins if necessary
    y_min, y_max = min(lower_bound_errors), max(upper_bound_errors)
    y_range = y_max - y_min
    if y_range < min_y_range:
        margin = min_y_range * 0.5
        y_min = np.mean([y_min, y_max]) - margin
        y_max = np.mean([y_min, y_max]) + margin
    else:
        margin = y_range * 0.1
        y_min -= margin
        y_max += margin

    # Plot results
    plt.figure(figsize=(10, 6))
    plt.plot(average_errors_df['week_before_target'], avg_errors_array, marker='o', color='b', linestyle='-', label='Average MAE', lw=2, markersize=6)
    plt.fill_between(average_errors_df['week_before_target'], lower_bound_errors, upper_bound_errors, color='b', alpha=0.2, label='95% Confidence Interval')
    plt.ylim([y_min, y_max])
    plt.xlabel('Weeks Before Target', fontsize=14)
    plt.ylabel('Average Mean Absolute Error', fontsize=14)
    plt.title(f'{method}: Average MAE over {weeks_before_target} Weeks Before Target Across {total_week_iterations} Iterations', fontsize=16)
    plt.gca().invert_xaxis()
    plt.gca().yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
    plt.grid(True, which='both', linestyle='--', linewidth=0.7, alpha=0.7)
    plt.legend(loc='upper right', fontsize=12)
    plt.tight_layout()
    plt.show()

In [9]:
# Function to evaluate predictions and plot error bars for supervised learning models
def evaluate_predictions(all_errors, method, weeks_before_target=8, total_week_iterations=20):
    min_y_range = 0.05
    average_errors, lower_bound_errors, upper_bound_errors = [], [], []

    for i in range(1, weeks_before_target + 1):
        errors_for_week = all_errors.get(i, [])
        if errors_for_week:
            average_error = np.mean(errors_for_week)
            sem = np.std(errors_for_week) / np.sqrt(len(errors_for_week))
            ci = 1.96 * sem  # 95% confidence interval

            # Store results
            average_errors.append({'week_before_target': i, 'average_mean_absolute_error': average_error})
            lower_bound_errors.append(average_error - ci)
            upper_bound_errors.append(average_error + ci)
    print(f"Method: {method} | MAEs {average_errors}")
    average_errors_df = pd.DataFrame(average_errors)
    avg_errors_array = average_errors_df['average_mean_absolute_error'].to_numpy()

    # Calculate y-limits and add margins if necessary
    y_min, y_max = min(lower_bound_errors), max(upper_bound_errors)
    y_range = y_max - y_min
    if y_range < min_y_range:
        margin = min_y_range * 0.5
        y_min = np.mean([y_min, y_max]) - margin
        y_max = np.mean([y_min, y_max]) + margin
    else:
        margin = y_range * 0.1
        y_min -= margin
        y_max += margin

    # Plot results
    plt.figure(figsize=(10, 6))
    plt.plot(average_errors_df['week_before_target'], avg_errors_array, marker='o', color='b', linestyle='-', label='Average MAE', lw=2, markersize=6)
    plt.fill_between(average_errors_df['week_before_target'], lower_bound_errors, upper_bound_errors, color='b', alpha=0.2, label='95% Confidence Interval')
    plt.ylim([y_min, y_max])
    plt.xlabel('Weeks Before Target', fontsize=14)
    plt.ylabel('Average Mean Absolute Error', fontsize=14)
    plt.title(f'{method}: Average MAE over {weeks_before_target} Weeks Before Target Across {total_week_iterations} Iterations', fontsize=16)
    plt.gca().invert_xaxis()
    plt.gca().yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
    plt.grid(True, which='both', linestyle='--', linewidth=0.7, alpha=0.7)
    plt.legend(loc='upper right', fontsize=12)
    plt.tight_layout()
    plt.show()

In [10]:
# Statistical Algorithms
def get_total_average_predictions(X_train, y_train, X_test):
    average_value = y_train.mean()
    predictions = np.full(X_test.shape[0], average_value)
    return pd.Series(predictions, index=X_test.index)

def get_last_occurrence_predictions(X_train, y_train, X_test):
    default_value = y_train.mean()
    last_occurrence_results = {}
    for item_id in X_test['item_id']:
        X_train_lookup = X_train[X_train['item_id'] == item_id]
        if not X_train_lookup.empty:
            max_menu_week = X_train_lookup['menu_week'].max()
            last_occurrence_results[item_id] = y_train[(X_train['item_id'] == item_id) & (X_train['menu_week'] == max_menu_week)].values[0]
        else:
            last_occurrence_results[item_id] = default_value
    return X_test['item_id'].map(last_occurrence_results)

def get_average_occurrence_predictions(X_train, y_train, X_test):
    default_value = y_train.mean()
    avg_occurrence_results = {}
    for item_id in X_test['item_id']:
        X_train_lookup = X_train[X_train['item_id'] == item_id]
        if not X_train_lookup.empty:
            avg_occurrence_results[item_id] = y_train[X_train['item_id'] == item_id].mean()
        else:
            avg_occurrence_results[item_id] = default_value
    return X_test['item_id'].map(avg_occurrence_results)


In [11]:
# Linear Regression
def get_linear_regression_model(X_train, y_train):
    model = LinearRegression()
    model.fit(X_train, y_train)
    return model

In [12]:
# LightGBM
def get_lightgbm_model(X_train, y_train):
    params = {
        'verbose': -1,
        'objective': 'regression',
        'metric': 'mae',
        'n_estimators': 100,
        'learning_rate': 0.2200044953698462,
        'max_depth': 18,
        'num_leaves': 669,
        'min_child_samples': 59,
        'min_split_gain': 1.4670533847983645e-06,
        'subsample': 0.7382953054809258,
        'colsample_bytree': 0.9648983195093089,
        'lambda_l1': 4.840165592010454e-07,
        'lambda_l2': 0.0025555820497953578,
        'random_state': 42,
        'n_jobs': -1,
    }
    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train)
    return model

In [13]:
# XGBoost
def get_xgboost_model(X_train, y_train):
    params = {
        'verbosity': 0,
        'objective': 'reg:logistic',
        'booster': 'gbtree',
        'eval_metric': 'mae',
        'n_estimators': 100,
        'learning_rate': 0.1759631363500291,
        'max_depth': 18,
        'min_child_weight': 2,
        'gamma': 0.0629948621294653,
        'subsample': 0.8156395766546685,
        'colsample_bytree': 0.5720051923599051,
        'colsample_bylevel': 0.8213911771088588,
        'lambda': 0.06733986476289976,
        'alpha': 0.8232226058749751,
        'random_state': 42,
        'n_jobs': -1,
    }
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train)
    return model

In [14]:
# Random Forest
def get_random_forest_model(X_train, y_train):
    params = {
        'n_estimators': 100,
        'max_depth': None,
        'min_samples_split': 4,
        'min_samples_leaf': 1,
        'max_features': 'log2',
        'max_leaf_nodes': None,
        'min_weight_fraction_leaf': 0.0,
        'max_samples': None,
        'criterion': 'friedman_mse',
        'min_impurity_decrease': 0.08,
        'random_state': 42,
        'n_jobs': -1,
    }
    model = RandomForestRegressor(**params)
    model.fit(X_train, y_train)
    return model

In [15]:
# LightGBM Ranker
def get_lgbm_ranker_model(X_train, y_train):
    group_train = X_train.groupby('menu_week').size().to_list() 
    params = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        'boosting_type': 'gbdt',
        'max_depth': 13,
        'learning_rate': 0.06732857519006756,
        'n_estimators': 623,
        'min_child_weight': 6.207324113550803,
        'min_child_samples': 51,
        'subsample': 0.772260146512238,
        'subsample_freq': 8,
        'colsample_bytree': 0.6959641369551797,
        'reg_alpha': 0.0013516164018592264,
        'reg_lambda': 0.002763584077083565,
        'importance_type': 'split',
        'label_gain': 5*list(range(max(group_train))),
        'verbose': -1
    }
    model = lgb.LGBMRanker(**params)
    model.fit(X_train, y_train, group=group_train)
    return model

In [16]:
# XGBoost Ranker
def get_xgboost_ranker_model(X_train, y_train):
    group_train = X_train.groupby('menu_week').size().to_list() 
    params = {
        'objective': 'rank:pairwise',
        'eval_metric': 'ndcg',
        'learning_rate': 0.1413861549700997,
        'max_depth': 6,
        'min_child_weight': 17,
        'gamma': 1.0529385943580223,
        'subsample': 0.9283230260401305,
        'colsample_bytree': 0.6587909355240208,
        'colsample_bylevel': 0.5063142658549241,
        'lambda': 0.008776031157446995,
        'alpha': 0.23436060259654692,
        'random_state': 42,
        'n_jobs': -1,
        'ndcg_exp_gain': False,
        'random_state': 42,
    }

    num_boost_round = 458

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtrain.set_group(group_train)

    model = xgb.train(params, dtrain, num_boost_round=num_boost_round)
    return model

In [17]:
# CatBoost Ranker
def get_catboost_ranker_model(X_train, y_train):
    group_train = X_train['menu_week']
    
    params = {
        'learning_rate': 0.3477239622054686,
        'depth': 9,
        'iterations': 100,
        'loss_function': 'PairLogit',
        'verbose': False,
        'random_seed': 42,
        'l2_leaf_reg': 9.360490681305208,
        'colsample_bylevel': 0.7863055932866824,
        'subsample': 0.9448577817655559,
        'bootstrap_type': 'Bernoulli',
        'eval_metric': 'NDCG',
    }

    model = cb.CatBoostRanker(**params)
    model.fit(X_train, y_train, group_id=group_train)
    return model

In [18]:
# RNN Model
def get_rnn_model(X_train, y_train, epochs=300, batch_size=16, units=64, time_steps=1):
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    
    # Reshaping data for RNN: (samples, time_steps, features)
    X_train_reshaped = np.reshape(X_train_scaled, (X_train_scaled.shape[0], time_steps, X_train_scaled.shape[1]))

    # Build the RNN model
    model = Sequential()
    model.add(Input(shape=(time_steps, X_train_scaled.shape[1])))
    model.add(SimpleRNN(units=units, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(SimpleRNN(units=units, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

    # Compile and fit model
    model.compile(optimizer='adam', loss='mean_absolute_error')
    model.fit(X_train_reshaped, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

    return model, scaler

# LSTM Model
def get_lstm_model(X_train, y_train, epochs=300, batch_size=16, units=64, time_steps=1):
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    
    # Reshaping data for LSTM: (samples, time_steps, features)
    X_train_reshaped = np.reshape(X_train_scaled, (X_train_scaled.shape[0], time_steps, X_train_scaled.shape[1]))

    # Build the LSTM model
    model = Sequential()
    model.add(Input(shape=(time_steps, X_train_scaled.shape[1])))
    model.add(LSTM(units=units, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=units, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

    # Compile and fit model
    model.compile(optimizer='adam', loss='mean_absolute_error')
    model.fit(X_train_reshaped, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

    return model, scaler

# Function to predict both neural network models and scale data
def nn_predict(trained_model, X_test, scaler, time_steps=1):
    X_test_scaled = scaler.transform(X_test)
    X_test_reshaped = np.reshape(X_test_scaled, (X_test_scaled.shape[0], time_steps, X_test_scaled.shape[1]))

    y_pred_values = trained_model.predict(X_test_reshaped, verbose=0)
    return y_pred_values

In [19]:
# Function to tune LightGBM Hyperparameters
def tune_lightgbm_hyperparameters(df, start_week, n_splits=5, n_trials=5):
    initial_data = df[df['menu_week'] < start_week]
    
    X = initial_data.drop(columns=['recipe_rank_percentile'])
    y = initial_data['recipe_rank_percentile']

    def objective(trial):
        param = {
            'verbosity': -1,
            'objective': 'regression',
            'metric': 'mae',
            'n_estimators': 100,
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0, log=True),
            'max_depth': trial.suggest_int('max_depth', 2, 20),
            'num_leaves': trial.suggest_int('num_leaves', 20, 800),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            'min_split_gain': trial.suggest_float('min_split_gain', 1e-8, 1.0, log=True),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
            'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
            'random_state': 42,
            'n_jobs': -1,
        }

        maes = []
        tscv = custom_time_series_cv(X, n_splits)
        for train_index, valid_index in tscv:
            X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

            # LightGBM model
            model = lgb.LGBMRegressor(**param)
            model.fit(X_train, y_train)
            
            preds = model.predict(X_valid)
            mae = mean_absolute_error(y_valid, preds)
            maes.append(mae)
        
        return np.mean(maes)

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)

    vis.plot_param_importances(study).show()
    vis.plot_slice(study).show()

    return study.best_params

In [20]:
# Function to tune XGBoost Hyperparameters
def tune_xgboost_hyperparameters(df, start_week, n_splits=5, n_trials=5):
    initial_data = df[df['menu_week'] < start_week]
    
    X = initial_data.drop(columns=['recipe_rank_percentile'])
    y = initial_data['recipe_rank_percentile']

    def objective(trial):
        param = {
            'verbosity': 0,
            'objective': 'reg:logistic',
            'booster': 'gbtree',
            'eval_metric': 'mae',
            'n_estimators': trial.suggest_categorical('n_estimators', [int(x) for x in np.linspace(100, 1000, num=10)]),
            'learning_rate': 0.1759631363500291,
            'max_depth': 18,
            'min_child_weight': 2,
            'gamma': 0.0629948621294653,
            'subsample': 0.8156395766546685,
            'colsample_bytree': 0.5720051923599051,
            'colsample_bylevel': 0.8213911771088588,
            'lambda': 0.06733986476289976,
            'alpha': 0.8232226058749751,
            'random_state': 42,
            'n_jobs': -1,
        }

        maes = []
        tscv = custom_time_series_cv(X, n_splits)
        for train_index, valid_index in tscv:
            X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
            model = xgb.XGBRegressor(**param)
            model.fit(X_train, y_train)
            preds = model.predict(X_valid)
            mae = mean_absolute_error(y_valid, preds)
            maes.append(mae)
        return np.mean(maes)

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)
    
    vis.plot_param_importances(study).show()
    vis.plot_slice(study).show()
    
    return study.best_params

In [21]:
# Function to tune Random Forest Hyperparameters
def tune_random_forest_hyperparameters(df, start_week, n_splits=5, n_trials=5):
    initial_data = df[df['menu_week'] < start_week]
    
    X = initial_data.drop(columns=['recipe_rank_percentile'])
    y = initial_data['recipe_rank_percentile']

    def objective(trial):
        param = {
            'n_estimators': 100,
            'max_depth': trial.suggest_int('max_depth', 10, 100),
            'min_samples_split': 2,
            'min_samples_leaf': 1,
            'max_features': 'log2',
            'max_leaf_nodes': None,
            'min_weight_fraction_leaf': 0.0,
            'max_samples': None,
            'criterion': 'friedman_mse',
            'min_impurity_decrease': 0.08,
            'random_state': 42,
            'n_jobs': -1,
        }

        maes = []
        tscv = custom_time_series_cv(X, n_splits)
        for train_index, valid_index in tscv:
            X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
            model = RandomForestRegressor(**param)
            model.fit(X_train, y_train)
            preds = model.predict(X_valid)
            preds = pd.Series(preds).rank(ascending=True, method='first') / len(preds)
            mae = mean_absolute_error(y_valid, preds)
            maes.append(mae)
        return np.mean(maes)

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)

    vis.plot_param_importances(study).show()
    vis.plot_slice(study).show()
    
    return study.best_params

In [22]:
# Function to evaluate statistical algorithms
def run_evaluation_algos(df, target_week_range, weeks_before_target, training_window=None, method='average_occurrence'):
    df = df.copy()
    total_week_iterations = len(target_week_range)
    all_errors = []
    num_models = 0
    training_window = max(target_week_range) + 1 if training_window is None else training_window
    
    method_dict = {
    'total_average': get_total_average_predictions,
    'last_occurrence': get_last_occurrence_predictions,
    'average_occurrence': get_average_occurrence_predictions,
    }

    df['recipe_rank'] = df.groupby('menu_week')['recipe_rank'].rank(method='first', pct=True, ascending=True)

    for max_week in target_week_range:
        menu_week_range = range(max_week - weeks_before_target, max_week)
        iteration_errors = []
        
        for menu_week_index in menu_week_range:
            start_time = time.time()
            num_models += 1

            train_df = df[(df['menu_week'] >= max(menu_week_index - training_window, 0)) & (df['menu_week'] <= menu_week_index)]
            test_df = df[df['menu_week'] == max_week]

            X_train = train_df.drop(columns=['recipe_rank'])
            y_train = train_df['recipe_rank']

            X_test = test_df.drop(columns=['recipe_rank'])
            y_test = test_df['recipe_rank']
            
            y_pred_values = method_dict[method](X_train, y_train, X_test)
            y_pred_values = pd.Series(y_pred_values).rank(ascending=True, method='average') / len(y_pred_values)

            mean_absolute_error_value = mean_absolute_error(y_test.values, y_pred_values.values)
            iteration_errors.append({'menu_week': menu_week_index, 'mean_absolute_error': mean_absolute_error_value})

            current_mean_error = np.mean([error['mean_absolute_error'] for errors in all_errors + [iteration_errors] for error in errors])

            time_taken = time.time() - start_time
            print(f"No. of models progress: {num_models}/{weeks_before_target*total_week_iterations} | Rolling MAE: {current_mean_error:.4f} | Estimated Time Left: {(1/60 * time_taken*(weeks_before_target*total_week_iterations - num_models)):.1f} minutes", end='\r', flush=True)

        all_errors.append(iteration_errors)
        
    evaluate_predictions_algos(all_errors, method, weeks_before_target, total_week_iterations)

In [23]:
# Function to evaluate supervised learning models
def run_evaluation(df, target_week_range, weeks_before_target, training_window=None, method='xgboost'):
    # Copy df to avoid changes to main df
    df = df.copy()
    # Initialise values for total week interations, training window size and pre-allocation for errors
    total_week_iterations = len(target_week_range)
    all_errors = {weeks_before: [] for weeks_before in range(1, weeks_before_target + 1)}
    training_window = max(target_week_range) + 1 if training_window is None else training_window

    # Initialise values for num/total models/inferences
    num_models = 0
    num_inferences = 0
    total_models = weeks_before_target + total_week_iterations - 1
    total_inferences = weeks_before_target * total_week_iterations

    # dict for methods and corresponding model functions
    method_dict = {
    'linear_regression': get_linear_regression_model,
    'lightgbm': get_lightgbm_model,
    'xgboost': get_xgboost_model,
    'random_forest': get_random_forest_model,
    'lgbm_ranker': get_lgbm_ranker_model,
    'xgboost_ranker': get_xgboost_ranker_model,
    'catboost_ranker': get_catboost_ranker_model,
    'rnn': get_rnn_model,
    'lstm': get_lstm_model,
    }

    # List of methods which require training on rank percentiles
    rank_percentile_models = ['linear_regression', 'lightgbm', 'xgboost', 'random_forest', 'catboost_ranker', 'rnn' ,'lstm']

    # Target feature engineering of rank percentile models
    if method in rank_percentile_models:
        df['recipe_rank'] = df.groupby('menu_week')['recipe_rank'].rank(method='first', pct=True, ascending=True)

    # Assign model function based on method input
    model_func = method_dict[method]
    
    # Loop over all menu weeks as the max training week
    for menu_week_index in range(min(target_week_range) - weeks_before_target, max(target_week_range)):
        num_models += 1
        start_time = time.time()

        # Filter training set for the current range of weeks within training window
        train_df = df[(df['menu_week'] >= max(menu_week_index - training_window, 0)) & (df['menu_week'] <= menu_week_index)]
        X_train = train_df.drop(columns=['recipe_rank'])
        y_train = train_df['recipe_rank']

        # Train the model only once for this menu_week_index (using scalar for nn models)
        if (method in ['rnn', 'lstm']):
            trained_model, scaler = model_func(X_train, y_train)
        else: 
            trained_model = model_func(X_train, y_train)  # Assuming return_model=True returns the trained model

        time_taken = time.time() - start_time

        # Test on each relevant max_week in the target_week_range
        for max_week in target_week_range:
            if max_week > menu_week_index >= max_week - weeks_before_target:
                num_inferences += 1
                # Filter test set for the current max_week
                test_df = df[df['menu_week'] == max_week]
                X_test = test_df.drop(columns=['recipe_rank'])
                y_test = test_df['recipe_rank']

                # Predict based on model type
                if (method == 'xgboost_ranker'):
                    dtest = xgb.DMatrix(X_test)
                    y_pred_values = trained_model.predict(dtest)
                elif (method in ['rnn', 'lstm']):
                    y_pred_values = nn_predict(trained_model, X_test, scaler)
                    y_pred_values = y_pred_values.flatten()
                else:
                    y_pred_values = trained_model.predict(X_test)

                # Calculate rank percentile of predictions
                y_pred_values = pd.Series(y_pred_values).rank(ascending=True, method='first') / len(y_pred_values)

                # Convert test values to percentiles too, if not initially converted and trained upon
                if method not in rank_percentile_models:
                    y_test = (y_test + 1)/len(y_test)

                # Calculate mean absolute error
                mean_absolute_error_value = mean_absolute_error(y_test.values, y_pred_values.values)
                weeks_before = max_week - menu_week_index
                all_errors[weeks_before].append(mean_absolute_error_value)
                current_mean_error = np.mean([error for errors in all_errors.values() for error in errors])

                print(f"No. of models progress: {num_models}/{total_models} | No. of inferences progress: {num_inferences}/{total_inferences} | Rolling MAE: {current_mean_error:.4f} | Estimated Time Left: {(1/60 * time_taken*(total_models - num_models)):.1f} minutes", end='\r')

    evaluate_predictions(all_errors, method, weeks_before_target, total_week_iterations)
    return current_mean_error

In [24]:
# Run evaluation to output shapley values
def run_evaluation_shap(df, target_week_range, weeks_before_target, training_window, method='xgboost'):
    df = df.copy()
    total_week_iterations = len(target_week_range)
    all_errors = {weeks_before: [] for weeks_before in range(1, weeks_before_target + 1)}
    training_window = max(target_week_range) + 1 if training_window is None else training_window

    # Dictionary to accumulate SHAP values across all weeks and iterations
    shap_values_accumulated = pd.DataFrame()

    num_models = 0
    num_inferences = 0
    total_models = weeks_before_target + total_week_iterations - 1
    total_inferences = weeks_before_target * total_week_iterations

    method_dict = {
        'linear_regression': get_linear_regression_model,
        'lightgbm': get_lightgbm_model,
        'xgboost': get_xgboost_model,
        'random_forest': get_random_forest_model,
        'lgbm_ranker': get_lgbm_ranker_model,
        'xgboost_ranker': get_xgboost_ranker_model,
        'catboost_ranker': get_catboost_ranker_model,
    }

    rank_percentile_models = ['linear_regression', 'lightgbm', 'xgboost', 'random_forest', 'catboost_ranker']

    if method in rank_percentile_models:
        df['recipe_rank'] = df.groupby('menu_week')['recipe_rank'].rank(method='first', pct=True, ascending=True)

    model_func = method_dict[method]
    
    for menu_week_index in range(min(target_week_range) - weeks_before_target, max(target_week_range)):
        num_models += 1
        start_time = time.time()

        train_df = df[(df['menu_week'] >= max(menu_week_index - training_window, 0)) & (df['menu_week'] <= menu_week_index)]
        X_train = train_df.drop(columns=['recipe_rank'])
        y_train = train_df['recipe_rank']

        trained_model = model_func(X_train, y_train)

        if method in ['xgboost', 'lightgbm', 'random_forest', 'xgboost_ranker', 'lgbm_ranker', 'catboost_ranker']:
            explainer = shap.TreeExplainer(trained_model)
        elif method == 'linear_regression':
            explainer = shap.LinearExplainer(trained_model, X_train)
        else:
            explainer = shap.KernelExplainer(trained_model.predict, X_train)

        time_taken = time.time() - start_time

        for max_week in target_week_range:
            if max_week > menu_week_index >= max_week - weeks_before_target:
                num_inferences += 1

                test_df = df[df['menu_week'] == max_week]
                X_test = test_df.drop(columns=['recipe_rank'])
                y_test = test_df['recipe_rank']

                if method in ['xgboost_ranker']:
                    dtest = xgb.DMatrix(X_test)
                    y_pred_values = trained_model.predict(dtest)
                elif method in ['lgbm_ranker', 'catboost_ranker']:
                    y_pred_values = trained_model.predict(X_test)
                else:
                    y_pred_values = trained_model.predict(X_test)

                y_pred_values = pd.Series(y_pred_values).rank(ascending=True, method='first') / len(y_pred_values)

                if method not in rank_percentile_models:
                    y_test = (y_test + 1)/len(y_test)

                shap_values = explainer.shap_values(X_train)
                shap_df = pd.DataFrame(shap_values, columns=X_test.columns)

                shap_values_accumulated = shap_values_accumulated.add(shap_df, fill_value=0)

                mean_absolute_error_value = mean_absolute_error(y_test.values, y_pred_values.values)
                weeks_before = max_week - menu_week_index
                all_errors[weeks_before].append(mean_absolute_error_value)
                current_mean_error = np.mean([error for errors in all_errors.values() for error in errors])

                print(f"No. of models progress: {num_models}/{total_models} | No. of inferences progress: {num_inferences}/{total_inferences} | Rolling MAE: {current_mean_error:.4f} | Estimated Time Left: {(1/60 * time_taken*(total_models - num_models)):.1f} minutes", end='\r')

    # Calculate the average SHAP values across all iterations
    avg_shap_values = shap_values_accumulated.abs().mean().sort_values(ascending=False)

    # Print the top 15 features based on average SHAP values across all iterations
    print("\nTop features based on average SHAP values across all weeks and iterations:")
    print(avg_shap_values.head(15))
    # Return average SHAP values for further analysis
    return avg_shap_values

In [25]:
# Training window tuning
training_window_range = range(100, 500, 50)
algorithms = [
    # {'name': 'Total Average', 'method': 'total_average'},
    # {'name': 'Last Occurrence', 'method': 'last_occurrence'},
    # {'name': 'Average Occurrence', 'method': 'average_occurrence'},
    # {'name': 'Linear Regression', 'method': 'linear_regression'},
    # {'name': 'LightGBM', 'method': 'lightgbm'},
    # {'name': 'XGBoost', 'method': 'xgboost'},
    # {'name': 'Random Forest', 'method': 'random_forest'},
    # {'name': 'LightGBM Ranker', 'method': 'lgbm_ranker'},
    # {'name': 'XGBoost Ranker', 'method': 'xgboost_ranker'},
    # {'name': 'CatBoost Ranker', 'method': 'catboost_ranker'}
]

# Dictionary to store the results for each algorithm
mae_results = {algo['name']: [] for algo in algorithms}

original_max_week = df['menu_week'].max() + 1

# Run the evaluation
total_week_iterations = 5
weeks_before_target = 8
target_week_range = range(original_max_week - total_week_iterations, original_max_week)

for algo in algorithms:
    for training_window in training_window_range:
        print(f"Running {algo['name']} with training window {training_window}")
        if algo['method'] in ['total_average', 'last_occurrence', 'average_occurrence']:
            mean_mae = run_evaluation_algos(df, target_week_range, weeks_before_target, training_window, method=algo['method'])
        else:
            mean_mae = run_evaluation(df, target_week_range, weeks_before_target, training_window, method=algo['method'])
        print(f"Mean MAE = {mean_mae}                                                                                                                                    ")
        print("")
        mae_results[algo['name']].append(mean_mae)

if any(len(mae_values) > 0 for mae_values in mae_results.values()):
    # Plot the results if there is data
    plt.figure(figsize=(10, 6))

    for algo_name, mae_values in mae_results.items():
        if mae_values:
            plt.plot(training_window_range, mae_values, label=algo_name)

    plt.title('Mean MAE vs Training Window Size for Different Algorithms')
    plt.xlabel('Training Window Size')
    plt.ylabel('Mean MAE')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

In [26]:
# Hyperparameter Tuning
original_max_week = df['menu_week'].max() + 1

# Run the evaluation
total_week_iterations = 52
weeks_before_target = 8
target_week_range = range(original_max_week - total_week_iterations, original_max_week)

# Tuning
n_splits = 3
n_trials = 150

start_week_for_tuning = original_max_week - total_week_iterations - weeks_before_target - n_splits

# best_params_lightgbm = tune_lightgbm_hyperparameters(df, start_week_for_tuning, n_splits=n_splits, n_trials=n_trials); print(best_params_lightgbm)
# best_params_xgboost = tune_xgboost_hyperparameters(df, start_week_for_tuning, n_splits=n_splits, n_trials=n_trials); print(best_params_xgboost)
# best_params_random_forest = tune_random_forest_hyperparameters(df, start_week_for_tuning, n_splits=n_splits, n_trials=n_trials); print(best_params_random_forest)

In [ ]:
# Model Execution
setback = 0
original_max_week = df['menu_week'].max() + 1 - setback

total_week_iterations = 52
weeks_before_target = 8
target_week_range = range(original_max_week - total_week_iterations, original_max_week)

# print("Total Average Algorithm:"); run_evaluation_algos(df, target_week_range, weeks_before_target, training_window=None, method='total_average'); print("\n")

# print("Last Occurrence Algorithm:"); run_evaluation_algos(df, target_week_range, weeks_before_target, training_window=None, method='last_occurrence'); print("\n")

# print("Average Occurrence Algorithm:"); run_evaluation_algos(df, target_week_range, weeks_before_target, training_window=120, method='average_occurrence'); print("\n")

# print("Linear Regression:"); run_evaluation(df, target_week_range, weeks_before_target, training_window=50, method='linear_regression'); print("\n")

# print("LightGBM:"); run_evaluation(df, target_week_range, weeks_before_target, training_window=100, method='lightgbm'); print("\n")

# print("XGBoost:"); run_evaluation(df, target_week_range, weeks_before_target, training_window=300, method='xgboost'); print("\n")

# print("Random Forest:"); run_evaluation(df, target_week_range, weeks_before_target, training_window=120, method='random_forest'); print("\n")

# print("LightGBM Ranker:") ; run_evaluation(df, target_week_range, weeks_before_target, training_window=180, method='lgbm_ranker'); print("\n")

# print("XGBoost Ranker:") ; run_evaluation(df, target_week_range, weeks_before_target, training_window=300, method='xgboost_ranker'); print("\n")

# print("CatBoost Ranker:") ; run_evaluation(df, target_week_range, weeks_before_target, training_window=250, method='catboost_ranker'); print("\n")

# print("RNN:") ; run_evaluation(df, target_week_range, weeks_before_target, training_window=500, method='rnn'); print("\n")

# print("LSTM:") ; run_evaluation(df, target_week_range, weeks_before_target, training_window=500, method='lstm'); print("\n")


In [28]:
# Combining all data from json file
with open('long_term_results.json', 'r') as f:
    long_term_results = json.load(f)

print(long_term_results)

# Calculate the mean of MAEs for each recipe (method)
mean_maes = {
    recipe: np.mean([item['average_mean_absolute_error'] for item in values])
    for recipe, values in long_term_results.items()
}

# Sort the methods by the mean of MAEs in descending order
sorted_methods = sorted(mean_maes.items(), key=lambda x: x[1], reverse=True)

# Initialize the plot
plt.figure(figsize=(15, 6))
colormap = plt.colormaps.get_cmap('tab20')
colors = colormap(np.linspace(0, 1, len(sorted_methods)))
for i, (recipe, _) in enumerate(sorted_methods):
    values = long_term_results[recipe]
    weeks = [item['week_before_target'] for item in values]
    errors = [item['average_mean_absolute_error'] for item in values]
    
    plt.plot(weeks, errors, marker='x', label=f"{recipe} (Mean MAE: {mean_maes[recipe]:.4f})", color=colors[i])

plt.xlabel('Week Before Target')
plt.ylabel('Average Mean Absolute Error')
plt.title('Combined Plot of Average MAEs Across Weeks for All Long Term Methods')
plt.gca().invert_xaxis()
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout(rect=[0, 0, 0.85, 1])
plt.show()